<a href="https://colab.research.google.com/github/Robert-Gomez-AI/HeartFailure/blob/main/M2U5_fase_4_Modelamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1A9dhO0gYByNJf9ya0SZmeCzZIeepRhsO" alt = "Encabezado MLDS" width = "100%">  </img>

# **Modelamiento y Validación**
---


## **0. Integrantes del equipo de trabajo**
---

Por favor incluya los nombres completos y número de identificación de los integrantes del equipo de trabajo:




1.      John Robert Gomez Pachón

2.  Yojhan Leandro Roldan Robles

3.   Daniela Mejia Castro



   


## Introducción

En el ámbito de la medicina, la predicción de la supervivencia de pacientes con insuficiencia cardíaca representa un desafío crucial para la atención médica. Un estudio destacado, titulado "Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone", publicado en la revista Circulation: Heart Failure, aborda esta problemática con un enfoque innovador y prometedor. En este estudio, se demostró que es posible predecir la supervivencia de pacientes con insuficiencia cardíaca utilizando únicamente dos variables clínicas: la creatinina sérica y la fracción de eyección del ventrículo izquierdo del corazón (FEVI).

Motivados por los resultados prometedores de este estudio, nos proponemos realizar un modelo basado en su metodología para predecir la supervivencia de pacientes con insuficiencia cardíaca. Para lograrlo, nuestro enfoque inicial será abordar el desafío del desbalance de datos en el conjunto de datos, un aspecto crucial para garantizar la calidad y la precisión del modelo. Posteriormente, implementaremos varios algoritmos de aprendizaje automático para construir modelos predictivos basados en las características clínicas mencionadas.

Además, emplearemos un sistema de búsqueda de grid de hiperparámetros para optimizar el rendimiento de nuestros modelos, ajustando los parámetros de los algoritmos de aprendizaje automático para obtener los mejores resultados posibles. Finalmente, evaluaremos exhaustivamente el rendimiento de nuestro modelo utilizando métricas de evaluación pertinentes, con el objetivo de validar su eficacia y su capacidad de generalización en la predicción de la supervivencia de pacientes con insuficiencia cardíaca.

En este proceso, nos basaremos en los hallazgos y la metodología del estudio mencionado como referencia clave, aprovechando su enfoque innovador y sus resultados significativos para contribuir al avance de la investigación y la práctica clínica en el campo de la medicina cardiovascular.

## **1. Selección y diseño de modelos**
---
El estudio mencionado aborda un problema de aprendizaje supervisado en el campo de la medicina, específicamente en el área de la salud cardiovascular. Este problema se centra en predecir la supervivencia de pacientes con insuficiencia cardíaca utilizando datos médicos disponibles en registros electrónicos. Dado que se trata de un problema de clasificación, el objetivo es asignar a cada paciente una etiqueta de clase que indique si sobrevivirá o no.

El conjunto de datos utilizado en el estudio consiste en registros médicos de 299 pacientes con insuficiencia cardíaca, recopilados en 2015. Los datos incluyen una variedad de características clínicas, como síntomas, características físicas y valores de pruebas de laboratorio, siendo las más relevantes la creatinina sérica y la fracción de eyección del corazón.

Para abordar este problema, se aplican técnicas de aprendizaje automático, específicamente clasificadores, que utilizan los datos del conjunto de entrenamiento para aprender un modelo que pueda predecir la supervivencia de los pacientes en función de sus características médicas. Además, se lleva a cabo un análisis de clasificación de características para identificar los factores de riesgo más importantes.

Los resultados del estudio muestran que es posible predecir la supervivencia de los pacientes con insuficiencia cardíaca utilizando únicamente la creatinina sérica y la fracción de eyección, lo que sugiere que estas dos características son altamente predictivas y suficientes para realizar una predicción precisa. Esta capacidad de predicción tiene implicaciones significativas en la práctica clínica, ya que puede ayudar a los médicos a identificar mejor a los pacientes en riesgo y tomar decisiones de tratamiento más informadas y precisas.

In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
np.random.seed(42)



# Validation and searching utils
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report




import matplotlib.pyplot as plt
import seaborn as sns


In [102]:
def balance_data(df, target, percent_difference):
    df0 = df[df[target] == 0]
    df1 = df[df[target] == 1]

    # Calcular el número objetivo de filas para cada clase
    target_rows = min(df0.shape[0], df1.shape[0]) + int(percent_difference * min(df0.shape[0], df1.shape[0]))

    # Tomar una cantidad de filas de cada clase que se ajuste al número objetivo
    df0_balanced = df0.sample(target_rows, replace=True) if df0.shape[0] > target_rows else df0
    df1_balanced = df1.sample(target_rows, replace=True) if df1.shape[0] > target_rows else df1

    # Concatenar los DataFrames balanceados
    dataframe = pd.concat([df0_balanced, df1_balanced], axis=0)

    # Obtener los índices de las filas que se mantuvieron en el DataFrame balanceado
    remaining_indices = df.index.difference(dataframe.index)
    remaining_data = df.loc[remaining_indices]

    return dataframe, remaining_data

In [199]:
df= pd.read_csv('/content/heart_failure_clinical_records_dataset.csv')

In [104]:
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [105]:
print("Class Percentage:")
df["DEATH_EVENT"].value_counts(normalize=True)

Class Percentage:


DEATH_EVENT
0    0.67893
1    0.32107
Name: proportion, dtype: float64

In [106]:
# df, remaining_data=  balance_data(df,'DEATH_EVENT',0.1)

In [107]:
X =df.drop('DEATH_EVENT',axis=1).values
y= df['DEATH_EVENT'].values

#X_r, y_r= remaining_data.drop('DEATH_EVENT',axis=1).values , remaining_data['DEATH_EVENT'].values

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
#X_r = scaler.fit_transform( X_r )

In [108]:
X_train,X_test,y_train,y_test= train_test_split(X,y, test_size=0.2)

In [109]:
y_train. shape

(239,)

In [110]:
#remaining_data

In [111]:
X_train.shape

(239, 12)

In [112]:
X_test

array([[0.54545455, 0.        , 0.07131921, 0.        , 0.39393939,
        0.        , 0.03139775, 0.24719101, 0.65714286, 1.        ,
        1.        , 0.87544484],
       [0.18181818, 1.        , 0.03508548, 0.        , 0.31818182,
        0.        , 0.40841314, 0.04494382, 0.77142857, 1.        ,
        1.        , 0.83985765],
       [0.09090909, 0.        , 0.30862465, 1.        , 0.24242424,
        0.        , 0.37446963, 0.06741573, 0.74285714, 1.        ,
        0.        , 0.44483986],
       [0.72727273, 1.        , 0.01275836, 0.        , 0.31818182,
        1.        , 0.43993211, 1.        , 0.57142857, 1.        ,
        1.        , 0.02135231],
       [0.03636364, 0.        , 0.0100791 , 1.        , 0.39393939,
        0.        , 0.25687962, 0.07865169, 0.77142857, 1.        ,
        0.        , 0.24911032],
       [0.18181818, 1.        , 0.13115591, 1.        , 0.24242424,
        0.        , 0.25081828, 0.02247191, 0.65714286, 0.        ,
        0.        ,

---**INGRESE SU RESPUESTA**---

## **2. Entrenamiento del Modelo y Selección de Hiperparámetros**
---

Este estudio aborda la predicción de la supervivencia en pacientes con insuficiencia cardíaca mediante aprendizaje automático, utilizando datos médicos como la creatinina sérica y la fracción de eyección del corazón. Para ello, emplearemos SVM, Regresión Logística, Decision Tree Classifier y KNeighbors Classifier, ajustando sus hiperparámetros con Grid Search. Estas predicciones tienen el potencial de mejorar la atención clínica, ayudando a los médicos a tomar decisiones más informadas sobre el tratamiento y manejo de la insuficiencia cardíaca.

## Support vector machine

In [113]:


hyperparameters = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 10]
}


model = SVC()

# Create the GridSearchCV object with the model and hyperparameters
grid_search = GridSearchCV(model, hyperparameters, cv=5)

# Fit the model using GridSearchCV and perform cross-validation
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf']})

In [114]:
# Get the best found hyperparameters
best_params = grid_search.best_params_

#  Create a new model with the best hyperparameters
svm = SVC(**best_params)

# Compute the cross-validation scores using the best model
cross_val_scores = cross_val_score(svm, X_train, y_train, cv=5)

# Print the average cross-validation score
print("Average cross-validation score:", cross_val_scores.mean())

#  Train the model using all data and the best hyperparameters
svm.fit(X_train, y_train)

Average cross-validation score: 0.8451241134751774


SVC(C=1, gamma=0.1, kernel='linear')

## Logistic regression

In [115]:
hyperparameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000,10000] ,
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}


model = LogisticRegression()

# Create the GridSearchCV object with the model and hyperparameters
grid_search = GridSearchCV(model, hyperparameters, cv=5)

# Fit the model using GridSearchCV and perform cross-validation
grid_search.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [116]:
# Get the best found hyperparameters
best_params = grid_search.best_params_

#  Create a new model with the best hyperparameters
Lg = LogisticRegression(**best_params)

#  Compute the cross-validation scores using the best model
cross_val_scores = cross_val_score(Lg, X_train, y_train, cv=5)

#  Print the average cross-validation score
print("Average cross-validation score:", cross_val_scores.mean())

#  Train the model using all data and the best hyperparameters
Lg.fit(X_train, y_train)

Average cross-validation score: 0.8618794326241135


LogisticRegression(C=1, penalty='l1', solver='saga')

In [117]:
hyperparameters = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


model = DecisionTreeClassifier()


grid_search = GridSearchCV(model, hyperparameters, cv=5)


grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [118]:
best_params = grid_search.best_params_


D3 = DecisionTreeClassifier(**best_params)


cross_val_scores = cross_val_score(D3, X_train, y_train, cv=5)


print("Average cross-validation score:", cross_val_scores.mean())


D3.fit(X_train, y_train)

Average cross-validation score: 0.8075354609929077


DecisionTreeClassifier(max_depth=5, min_samples_split=5)

In [119]:


hyperparameters = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

model = KNeighborsClassifier()


grid_search = GridSearchCV(model, hyperparameters, cv=5)


grid_search.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']})

In [120]:


best_params = grid_search.best_params_


Knn = KNeighborsClassifier(**best_params)



print("Average cross-validation score:", cross_val_scores.mean())


Knn.fit(X_train, y_train)



Average cross-validation score: 0.8075354609929077


KNeighborsClassifier(metric='euclidean', n_neighbors=7)

## **3. Evaluación del modelo**
---

En esta sección debe reportar el desempeño del modelo sobre la partición de datos de pruebas (test). Considere que dispone de las siguientes métricas:

- **Clasificación**: accuracy, precision, recall, f1-score, AUC, matriz de confusión, etc.
- **Regresión**: $r^2$, error cuadrático medio, error absoluto medio, etc.
- **Agrupamiento**: coeficiente de silueta, índice de Davies-Bouldin, etc.

Los resultados se deben presentar usando tablas y figuras además del análisis detallado respectivo.

In [121]:
def evaluate_model(model, X, y):

    y_pred = model.predict(X)

    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1-Score': f1_score(y, y_pred)
    }

    return metrics


In [127]:
models = [svm,Knn,Lg, D3]
model_type=['Support Vector Machine','K-Nearest Neighbors','Logistic Regression','Decision three']
count=0
for model in models:

    print(model_type[count])
    evaluation=evaluate_model(model,X_test,y_test)
    for key in evaluation:
        print (f'{key}:{evaluation[key]}')
    count+=1
    print('\n')


Support Vector Machine
Accuracy:0.75
Precision:0.9166666666666666
Recall:0.44
F1-Score:0.5945945945945945


K-Nearest Neighbors
Accuracy:0.5833333333333334
Precision:0.5
Recall:0.08
F1-Score:0.13793103448275865


Logistic Regression
Accuracy:0.7833333333333333
Precision:0.9285714285714286
Recall:0.52
F1-Score:0.6666666666666666


Decision three
Accuracy:0.7166666666666667
Precision:0.6818181818181818
Recall:0.6
F1-Score:0.6382978723404256




Después de evaluar diversos modelos de machine learning para predecir cardiopatías, la precisión emerge como una métrica de alta relevancia, especialmente en el contexto de la salud, donde se busca minimizar los falsos positivos y asegurar un diagnóstico preciso.

Al analizar los resultados de los diferentes modelos, la Regresión Logística se destaca como el mejor modelo en términos de precisión (0.928), interpretabilidad y capacidad para extraer la probabilidad. Esto implica que aproximadamente el 92.8% de las predicciones positivas realizadas por este modelo son correctas, lo que sugiere una capacidad excepcional para identificar pacientes con cardiopatías.

Además, la Regresión Logística ofrece una ventaja única en términos de interpretabilidad, ya que proporciona coeficientes para cada característica que pueden ser fácilmente interpretados en términos de su contribución a la predicción. Esta interpretación se ve respaldada por el análisis de feature ranking realizado en el estudio, que empleó pruebas como el test U de Mann–Whitney, el coeficiente de correlación de Pearson y el test de chi-cuadrado para evaluar la asociación de cada característica con el objetivo. Asimismo, la capacidad de extraer probabilidades de la clase objetivo proporciona información valiosa para la toma de decisiones clínicas.

Aunque la Máquina de Vectores de Soporte (Support Vector Machine) también exhibe una alta precisión (0.917), su capacidad de interpretación y la extracción de probabilidades son más limitadas en comparación con la Regresión Logística.

En conclusión, basándonos en la relevancia de la precisión, la interpretabilidad y la capacidad para extraer probabilidades, la Regresión Logística parece ser el modelo más adecuado para predecir cardiopatías en este contexto específico. Sin embargo, es importante tener en cuenta que la selección del modelo también puede depender de otros factores, como el tiempo de entrenamiento y la facilidad de implementación en entornos clínicos.

## **4. Aplicación del modelo**
---

Los diagnósticos se realizan en bloque y se almacenan junto con las probabilidades en un dataframe para facilitar la gestión y el análisis de la información. Al agrupar los diagnósticos en bloque, se puede procesar de manera eficiente un gran volumen de datos de pacientes, lo que es fundamental en entornos clínicos donde se realizan múltiples diagnósticos simultáneamente. Además, almacenar tanto el diagnóstico como la probabilidad asociada en un dataframe permite una fácil visualización y comparación de los resultados, lo que facilita la toma de decisiones clínicas informadas y la identificación de patrones o tendencias en los datos de diagnóstico.

Utilizar un modelo de regresión logística para generar estos diagnósticos ofrece la ventaja adicional de su capacidad de generalización. La regresión logística puede aprender patrones y relaciones complejas en los datos de entrenamiento y aplicar ese conocimiento para hacer predicciones precisas en nuevos conjuntos de datos. Esto significa que el modelo puede adaptarse a diferentes condiciones y contextos clínicos, lo que aumenta su utilidad y relevancia en diversas situaciones médicas. Además, la regresión logística proporciona una medida de incertidumbre a través de las probabilidades asociadas a cada diagnóstico, lo que permite una evaluación más completa y precisa de la confiabilidad de las predicciones realizadas.

In [193]:
def predict(X):
  X= X.values
  scaler = MinMaxScaler()
  # Definir el diccionario de resultados
  results = {
      "Predicción": list(Lg.predict(X)),
      "Probabilidad": [prob[1] for prob in Lg.predict_proba(X)]  # Tomar solo las probabilidades de la clase positiva
  }

  # Convertir el diccionario en un DataFrame
  results_df = pd.DataFrame(results)
  return results_df

In [194]:
X_produccion = remaining_data.drop('DEATH_EVENT',axis=1)

In [195]:
data= predict(X_produccion)

In [196]:
data

,Predicción,Probabilidad
0,0,2.793014e-52
1,0,2.919090e-113
2,0,5.793154e-80
3,0,1.755305e-106
4,0,1.197596e-130
...,...,...
120,0,0.000000e+00
121,0,0.000000e+00
122,0,0.000000e+00
123,0,0.000000e+00


In [200]:
prediction_all=predict(df.drop('DEATH_EVENT',axis=1))

In [201]:
prediction_all

,Predicción,Probabilidad
0,1,9.999930e-01
1,0,5.116345e-45
2,0,1.907172e-11
3,0,1.160439e-24
4,0,1.749934e-08
...,...,...
294,0,0.000000e+00
295,0,0.000000e+00
296,0,0.000000e+00
297,0,0.000000e+00


from matplotlib import pyplot as plt
prediction_all['Predicción'].plot(kind='hist', bins=20, title='Predicción')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
prediction_all['Probabilidad'].plot(kind='hist', bins=20, title='Probabilidad')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
prediction_all.plot(kind='scatter', x='Predicción', y='Probabilidad', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
prediction_all['Predicción'].plot(kind='line', figsize=(8, 4), title='Predicción')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
prediction_all['Probabilidad'].plot(kind='line', figsize=(8, 4), title='Probabilidad')
plt.gca().spines[['top', 'right']].set_visible(False)

## **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistente docente :**
  * [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandrasuperlano/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandrasuperlano/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*